In [1]:
%pip install mediapipe
%pip install opencv-python

import cv2
import mediapipe as mp
import math
import random
import time

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# 原有的角度計算函數
def vector_2d_angle(v1, v2):
    v1_x = v1[0]
    v1_y = v1[1]
    v2_x = v2[0]
    v2_y = v2[1]
    try:
        angle_ = math.degrees(math.acos((v1_x*v2_x+v1_y*v2_y)/(((v1_x**2+v1_y**2)**0.5)*((v2_x**2+v2_y**2)**0.5))))
    except:
        angle_ = 180
    return angle_

# 原有的手指角度計算函數
def hand_angle(hand_):
    angle_list = []
    # thumb 大拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[2][0])),(int(hand_[0][1])-int(hand_[2][1]))),
        ((int(hand_[3][0])- int(hand_[4][0])),(int(hand_[3][1])- int(hand_[4][1])))
        )
    angle_list.append(angle_)
    # index 食指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])-int(hand_[6][0])),(int(hand_[0][1])- int(hand_[6][1]))),
        ((int(hand_[7][0])- int(hand_[8][0])),(int(hand_[7][1])- int(hand_[8][1])))
        )
    angle_list.append(angle_)
    # middle 中指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[10][0])),(int(hand_[0][1])- int(hand_[10][1]))),
        ((int(hand_[11][0])- int(hand_[12][0])),(int(hand_[11][1])- int(hand_[12][1])))
        )
    angle_list.append(angle_)
    # ring 無名指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[14][0])),(int(hand_[0][1])- int(hand_[14][1]))),
        ((int(hand_[15][0])- int(hand_[16][0])),(int(hand_[15][1])- int(hand_[16][1])))
        )
    angle_list.append(angle_)
    # pink 小拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[18][0])),(int(hand_[0][1])- int(hand_[18][1]))),
        ((int(hand_[19][0])- int(hand_[20][0])),(int(hand_[19][1])- int(hand_[20][1])))
        )
    angle_list.append(angle_)
    return angle_list

# 原有的手勢判斷函數，但只保留數字判斷
def hand_pos(finger_angle):
    f1 = finger_angle[0]   # 大拇指角度
    f2 = finger_angle[1]   # 食指角度
    f3 = finger_angle[2]   # 中指角度
    f4 = finger_angle[3]   # 無名指角度
    f5 = finger_angle[4]   # 小拇指角度

    # 只判斷數字手勢
    if f1>=50 and f2>=50 and f3>=50 and f4>=50 and f5>=50:
        return '0'
    elif f1>=50 and f2<50 and f3>=50 and f4>=50 and f5>=50:
        return '1'
    elif f1>=50 and f2<50 and f3<50 and f4>=50 and f5>=50:
        return '2'
    elif f1>=50 and f2<50 and f3<50 and f4<50 and f5>50:
        return '3'
    elif f1>=50 and f2<50 and f3<50 and f4<50 and f5<50:
        return '4'
    elif f1<50 and f2<50 and f3<50 and f4<50 and f5<50:
        return '5'
    elif f1<50 and f2>=50 and f3>=50 and f4>=50 and f5<50:
        return '6'
    elif f1<50 and f2<50 and f3>=50 and f4>=50 and f5>=50:
        return '7'
    elif f1<50 and f2<50 and f3<50 and f4>=50 and f5>=50:
        return '8'
    elif f1<50 and f2<50 and f3<50 and f4<50 and f5>=50:
        return '9'
    else:
        return ''

def main():
    cap = cv2.VideoCapture(0)
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    lineType = cv2.LINE_AA

    # Game variables
    correct_count = 0
    target_number = None
    show_gift = False
    answer_confirmed = False
    answer_display_time = 0
    wrong_answer_confirmed = False  # New: track if wrong answer was confirmed
    stable_frames = 0  # New: count how many frames the same number appears
    last_detected_number = None  # New: keep track of the last detected number
    REQUIRED_STABLE_FRAMES = 15  # New: number of frames required to confirm an answer

    with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

        if not cap.isOpened():
            print("Cannot open camera")
            exit()
        
        w, h = 640, 480

        while True:
            ret, img = cap.read()
            img = cv2.resize(img, (w,h))
            if not ret:
                print("Cannot receive frame")
                break
            
            if target_number is None:
                target_number = random.randint(0, 9)
            
            img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img2)

            # Display target and streak
            cv2.putText(img, f"Target: {target_number}", (30, 50), 
                       fontFace, 1, (255, 255, 255), 2, lineType)
            cv2.putText(img, f"Streak: {correct_count}", (30, 90), 
                       fontFace, 1, (255, 255, 255), 2, lineType)

            # if show_gift:
            #     cv2.putText(img, "", (w//2-50, h//2), 
            #                fontFace, 2, (255, 255, 255), 2, lineType)

            if results.multi_hand_landmarks:
                hand_numbers = []
                
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        img,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())

                    finger_points = []
                    for i in hand_landmarks.landmark:
                        x = i.x*w
                        y = i.y*h
                        finger_points.append((x,y))
                    
                    if finger_points:
                        finger_angle = hand_angle(finger_points)
                        hand_number = hand_pos(finger_angle)
                        if hand_number:
                            hand_numbers.append(hand_number)
                
                # Process hand gestures only if not in confirmed state
                if hand_numbers and not answer_confirmed and not wrong_answer_confirmed:
                    detected_numbers = sorted([int(num) for num in hand_numbers if num.isdigit()])
                    
                    # Show detected numbers
                    for i, num in enumerate(detected_numbers):
                        cv2.putText(img, str(num), (30 + i*60, 180), 
                                  fontFace, 2, (0, 255, 0), 2, lineType)
                    
                    # Get the first detected number (if any)
                    current_number = detected_numbers[0] if detected_numbers else None
                    
                    # Check if the number is stable
                    if current_number == last_detected_number:
                        stable_frames += 1
                    else:
                        stable_frames = 0
                        
                    last_detected_number = current_number
                    
                    # Only confirm answer after stable frames threshold
                    if stable_frames >= REQUIRED_STABLE_FRAMES:
                        if target_number in detected_numbers:
                            answer_confirmed = True
                            answer_display_time = time.time()
                            correct_count += 1
                            if correct_count >= 3:
                                show_gift = True
                                cv2.putText(img, "Wonderful!", (w//2-150, h//2-50), 
                                          fontFace, 1, (0, 255, 0), 2, lineType)                        
                                correct_count = 0
                        else:
                            wrong_answer_confirmed = True
                            answer_display_time = time.time()
                            # correct_count = 0
                            show_gift = False
                
                # Handle confirmed correct answer
                if answer_confirmed:
                    cv2.putText(img, "Correct!", (w//2-100, h//2), 
                              fontFace, 2, (0, 255, 0), 2, lineType)
                    
                    if time.time() - answer_display_time >= 1:
                        target_number = random.randint(0, 9)
                        answer_confirmed = False
                        stable_frames = 0
                
                # Handle confirmed wrong answer
                if wrong_answer_confirmed:
                    cv2.putText(img, "Try Again!", (w//2-100, h//2), 
                              fontFace, 2, (0, 0, 255), 2, lineType)
                    
                    if time.time() - answer_display_time >= 1:
                        target_number = random.randint(0, 9)
                        wrong_answer_confirmed = False
                        stable_frames = 0

            cv2.imshow('Hand Gesture Game', img)
            if cv2.waitKey(5) == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


I0000 00:00:1730916918.773493 13116118 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1730916918.791422 13116585 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730916918.799701 13116585 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730916919.107627 13116581 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/Users/yc97463/Documents/projects/gdscndhu23-certificate-issuer/crop/myenv/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead